In [ ]:
import pandas as pd
excel_path = 'data/map/namdinh/data.xlsx'
xls = pd.ExcelFile(excel_path)
all_data_with_sheet_name = []

# Iterate through each sheet, add a column for 'varname' to store the sheet name, and then append to the list
for sheet_name in xls.sheet_names:
    df_sheet = pd.read_excel(xls, sheet_name=sheet_name)
    # For each sheet, melt and then add the sheet name as 'varname'
    df_sheet_melted = df_sheet.melt(id_vars=["NAME"], var_name="YEAR", value_name="value")
    df_sheet_melted["varname"] = sheet_name
    all_data_with_sheet_name.append(df_sheet_melted)

# Concatenate all the melted DataFrames with the sheet name included
long_df_with_varname = pd.concat(all_data_with_sheet_name, ignore_index=True)

long_df_with_varname

In [ ]:
# Separate the DataFrames based on 'varname' for 'ph' and 'hh' using 'contains'
df_ph = long_df_with_varname[long_df_with_varname['varname'].str.contains('ph')].copy()
df_hh = long_df_with_varname[long_df_with_varname['varname'].str.contains('hh')].copy()

# Perform operations directly on the copy to avoid SettingWithCopyWarning
df_ph['YEAR'] = df_ph['YEAR']
df_hh['YEAR'] = df_hh['YEAR']

# Merge the 'ph' and 'hh' DataFrames on 'NAME' and 'year'
df_merged = pd.merge(df_ph, df_hh, on=['NAME', 'YEAR'], suffixes=('_ph', '_hh'))

# Calculate the new variable 'ph/hh * 100'
df_merged['perph'] = (df_merged['value_ph'] / df_merged['value_hh'])

# Create a DataFrame for the new variable
df_new_var = df_merged[['NAME', 'YEAR', 'perph']].copy()
df_new_var['varname'] = 'perph'

# Ensure the original DataFrame is not just a view to avoid the warning
long_df_with_varname_filtered = long_df_with_varname[~long_df_with_varname['varname'].str.contains('ph|hh')].copy()

# Concatenate the DataFrame with the new variable to the filtered original DataFrame
final_df = pd.concat([long_df_with_varname_filtered, df_new_var[['NAME', 'YEAR', 'varname', 'perph']].rename(columns={'perph': 'value'})], ignore_index=True)

# Note: Ensure to use '.copy()' when slicing DataFrames to avoid the SettingWithCopyWarning when setting new values.


In [ ]:
final_df

In [ ]:
pivot_df = final_df.pivot_table(index=['NAME', 'YEAR'], columns='varname', values='value').reset_index()

In [ ]:
pivot_df

In [ ]:
pivot_df['NAME'] = pivot_df['NAME'].str.replace(' ','').str.replace('Thịtrấn','').str.replace('Phường','')

In [ ]:
pivot_df